In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import os
from PIL import Image as Pimage
import cv2
from sklearn.utils import shuffle
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50,preprocess_input
from keras.layers import Activation,Convolution2D,Flatten,Input,add

In [2]:
df_labels = pd.read_csv('./labels.csv')
train_image_path = './train/'

In [76]:
print df_labels[:5]['id']

0    000bec180eb18c7604dcecc8fe0dba07
1    001513dfcb2ffafc82cccf4d8bbaba97
2    001cdf01b096e06d78e9e5112d419397
3    00214f311d5d2247d5dfe4fe24b2303d
4    0021f9ceb3235effd7fcde7f7538ed62
Name: id, dtype: object


In [4]:
im = cv2.imread("abc.tiff")
print type(im)

<type 'NoneType'>


In [5]:
images = []
for ix in os.listdir(train_image_path):
    temp = Pimage.open(train_image_path+ix,mode='r')
    image = np.array(temp)
    images.append(image)
    temp.close()

In [54]:
#to find the minimum possible shape of the input image

print  images[5].shape
min_x = 1000
min_y = 1000
for ix in images:
    x,y,z = ix.shape
    if x < min_x:
        min_x = x
    if y < min_y:
        min_y = y
print min_x,min_y,"--> thats could be the shape of minimum image"

(375, 500, 3)
102 97 --> thats could be the shape of minimum image


In [65]:
#preprocessing to make every image of shape 224,224,3
#122,127 were chosen because of corresponding values of min_x,min_y

new_images = []

for ix in images:
    x,y,z = ix.shape
    constant= cv2.copyMakeBorder(ix,0,122,0,127,cv2.BORDER_CONSTANT,value=0)
    temp = constant[:224,:224,:3]
    new_images.append(temp)

In [78]:
print len(new_images)
print new_images[0].shape
print new_images[5].shape

10222
(224, 224, 3)
(224, 224, 3)


In [7]:
image_names = []
for ix in os.listdir(train_image_path):
    image_names.append(ix)

In [8]:
print image_names[:10]

['000bec180eb18c7604dcecc8fe0dba07.jpg', '001513dfcb2ffafc82cccf4d8bbaba97.jpg', '001cdf01b096e06d78e9e5112d419397.jpg', '00214f311d5d2247d5dfe4fe24b2303d.jpg', '0021f9ceb3235effd7fcde7f7538ed62.jpg', '002211c81b498ef88e1b40b9abf84e1d.jpg', '00290d3e1fdd27226ba27a8ce248ce85.jpg', '002a283a315af96eaea0e28e7163b21b.jpg', '003df8b8a8b05244b1d920bb6cf451f9.jpg', '0042188c895a2f14ef64a918ed9c7b64.jpg']


In [9]:
#images have all train images

#image_names has corresponding names of train images name
# train_labels contains labels for train images
#new_images has processed images

In [10]:
print df_labels

                                     id                        breed
0      000bec180eb18c7604dcecc8fe0dba07                  boston_bull
1      001513dfcb2ffafc82cccf4d8bbaba97                        dingo
2      001cdf01b096e06d78e9e5112d419397                     pekinese
3      00214f311d5d2247d5dfe4fe24b2303d                     bluetick
4      0021f9ceb3235effd7fcde7f7538ed62             golden_retriever
5      002211c81b498ef88e1b40b9abf84e1d           bedlington_terrier
6      00290d3e1fdd27226ba27a8ce248ce85           bedlington_terrier
7      002a283a315af96eaea0e28e7163b21b                       borzoi
8      003df8b8a8b05244b1d920bb6cf451f9                      basenji
9      0042188c895a2f14ef64a918ed9c7b64           scottish_deerhound
10     004396df1acd0f1247b740ca2b14616e            shetland_sheepdog
11     0067dc3eab0b3c3ef0439477624d85d6                 walker_hound
12     00693b8bc2470375cc744a6391d397ec                  maltese_dog
13     006cc3ddb9dc1bd827479569fcd

In [11]:
train_labels = df_labels['breed']

In [12]:
# print train_labels
print train_labels.shape
print train_labels[:5]

(10222,)
0         boston_bull
1               dingo
2            pekinese
3            bluetick
4    golden_retriever
Name: breed, dtype: object


In [13]:
# print len(final_labels)
# final_labels = np.array(final_labels)
# print final_labels.shape

In [14]:
le = LabelEncoder()
final_train_labels = df_labels.apply(le.fit_transform)

In [15]:
 final_train_labels = final_train_labels['breed']

In [16]:
 np.unique(final_train_labels,return_counts=True)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119]),
 array([ 80, 116,  86, 107,  74,  78, 102, 110,  82, 105,  89, 114,  77,
        102,  85,  85,  72,  91,  75,  87,  86,  75,  67,  66,  73,  75,
        106,  76,  83,  71,  93,  80,  74,  87,  72,  89,  76,  80,  74,
         86,  83,  75, 115,  66,  72,  70,  69,  75,  69,  67,  81,  75,
        111,  82,  82,  91

In [17]:
Y = np_utils.to_categorical(final_train_labels)

In [18]:
print train_labels.shape
print images[7].shape

(10222,)
(227, 231, 3)


In [79]:
print train_labels.shape
print len(np.unique(train_labels,return_counts=False)),"--> number of output labels"

(10222,)
120 --> number of output labels


In [74]:
split = int(0.80*train_labels.shape[0])
x_train = new_images[:split]
y_train = Y[:split]
x_test = new_images[split:]
y_test = Y[split:]
print split
print type(x_train)
# print x_train.shape,y_train.shape
# print x_test.shape,y_test.shape

8177
<type 'list'>


In [34]:
image_input = Input(shape=(224,224,3))
model = ResNet50(weights='imagenet',include_top=True,input_tensor=image_input)

In [35]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [41]:
last_layer = model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(256,activation='relu',name='fc1')(x)
out = Dense(120,activation='softmax',name='output')(x)

custom_res = Model(image_input,out)

for layer in custom_res.layers[:-3]:
    layer.trainable = False

In [43]:
custom_res.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [72]:
custom_res.fit(x_train,y_train,epochs=2,verbose=2,validation_data=(x_test,y_test))

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 8177 arrays: [array([[[109,  46,   0],
        [153,  91,  40],
        [180, 120,  68],
        ..., 
        [130, 101,  69],
        [115,  78,  62],
        [125,  81,  80]],

       [[153,  92,  47],
        ...